In [1]:
import numpy as np
import pandas as pd
import warnings
import itertools
from tqdm import tqdm_notebook
from scipy.stats import spearmanr
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv(f'data/train.csv')
TARGETS = train.columns[11:]

for col in TARGETS:
    train[col] = train[col].rank(method="average")
train[TARGETS] = MinMaxScaler().fit_transform(train[TARGETS])
y = train[TARGETS].values
ix = np.where(train.groupby("question_body")["host"].transform("count") == 1)[0]

y_use = pd.read_csv('oofs/use_oof.csv').values
y_albert = pd.read_csv('oofs/siamese_albert_1_comb_oofs.csv', index_col=0).values
y_bert = pd.read_csv('oofs/siamese_bert_6_oofs.csv', index_col=0).values
y_roberta = pd.read_csv('oofs/siamese_roberta_1_comb_oofs.csv', index_col=0).values
y_xlnet = pd.read_csv('oofs/siamese_xlnet_1_comb_oofs.csv', index_col=0).values
# y_xlnet = pd.read_csv('oofs/siamese_xlnet_2_comb_oofs.csv', index_col=0).values

preds = [y_use, y_roberta, y_bert, y_xlnet, y_albert]
n_models = len(preds)

In [3]:
def mae(y, y_pred):
    return -np.mean(np.abs(y - y_pred))


def evaluate(y, y_pred, verbose=False, exclude=[]):
    score = 0
    for i in range(y.shape[1]):
        if i not in exclude:
            col_score = spearmanr(y[:, i], y_pred[:, i])[0]
            if verbose:
                print(TARGETS[i], np.round(col_score, 3))
            score += col_score/(y.shape[1]-len(exclude))
    return np.round(score, 4)


ds = [4, 8, 16, 32, 64, None]
ws = [0, 1, 2, 4]


def scale(x, d):
    if d:
        return (x//(1/d))/d
    return x

dws5 = list(itertools.product(ds, ds, *(n_models*[ws])))


def transform(preds, params, c):
    (d_global, d_local), ws = params[:2], params[2:]
    y_temp = 0
    for pred, w in zip(preds, ws):
        y_temp += w * scale(pred[:, c], d_local)
    y_temp /= sum(ws)
    y_temp = scale(y_temp, d_global)
    return y_temp


mae_weight = 0.5


def f(c):
    max_score = spearmanr(y[:, c], y_roberta[:, c])[0] 
    max_score -= mae_weight * mae(y[:, c], y_roberta[:, c])
    best_index = -1
    for i, params in enumerate(dws5):
        if sum(params[2:]) == 0:
            continue
        y_temp = transform(preds, params, c)
        score = spearmanr(y[:, c], y_temp)[0] 
        score -= mae_weight * mae(y[:, c], y_temp)
        if score > max_score:
            max_score = score
            best_index = i
            
    return best_index

In [4]:
import multiprocessing

pool = multiprocessing.Pool(15)
out = pool.map(f, range(30))

In [5]:
y_combined = np.zeros(y.shape)

for c in range(30):
    print(c, TARGETS[c], dws5[out[c]])
    y_combined[:, c] = transform(preds, dws5[out[c]], c)

0 question_asker_intent_understanding (16, 16, 1, 2, 2, 2, 2)
1 question_body_critical (16, 4, 4, 1, 4, 1, 4)
2 question_conversational (4, 32, 1, 4, 0, 1, 1)
3 question_expect_short_answer (16, 16, 2, 4, 1, 1, 4)
4 question_fact_seeking (4, 4, 1, 0, 2, 2, 1)
5 question_has_commonly_accepted_answer (4, None, 1, 4, 2, 1, 4)
6 question_interestingness_others (16, 4, 1, 1, 2, 1, 2)
7 question_interestingness_self (8, 8, 4, 2, 4, 1, 4)
8 question_multi_intent (None, None, 4, 2, 2, 2, 1)
9 question_not_really_a_question (16, 16, 1, 0, 2, 2, 0)
10 question_opinion_seeking (16, 4, 1, 2, 1, 2, 1)
11 question_type_choice (16, 4, 1, 1, 0, 1, 2)
12 question_type_compare (4, None, 0, 4, 1, 4, 0)
13 question_type_consequence (4, 8, 0, 1, 0, 2, 0)
14 question_type_definition (8, 8, 0, 4, 1, 0, 0)
15 question_type_entity (4, 8, 2, 1, 0, 4, 0)
16 question_type_instructions (4, None, 2, 4, 0, 1, 1)
17 question_type_procedure (None, None, 1, 2, 0, 2, 1)
18 question_type_reason_explanation (16, 4, 2, 4, 

In [6]:
evaluate(y, y_combined, True)

question_asker_intent_understanding 0.406
question_body_critical 0.68
question_conversational 0.512
question_expect_short_answer 0.323
question_fact_seeking 0.375
question_has_commonly_accepted_answer 0.486
question_interestingness_others 0.367
question_interestingness_self 0.525
question_multi_intent 0.609
question_not_really_a_question 0.187
question_opinion_seeking 0.492
question_type_choice 0.778
question_type_compare 0.566
question_type_consequence 0.333
question_type_definition 0.669
question_type_entity 0.623
question_type_instructions 0.794
question_type_procedure 0.385
question_type_reason_explanation 0.683
question_type_spelling 0.608
question_well_written 0.532
answer_helpful 0.287
answer_level_of_information 0.477
answer_plausible 0.18
answer_relevance 0.189
answer_satisfaction 0.391
answer_type_instructions 0.772
answer_type_procedure 0.327
answer_type_reason_explanation 0.694
answer_well_written 0.252


0.4835

In [9]:
evaluate(y, y_combined, False, exclude=[9,19])

0.4896

In [8]:
[dws5[out[c]] for c in range(30)]

[(16, 16, 1, 2, 2, 2, 2),
 (16, 4, 4, 1, 4, 1, 4),
 (4, 32, 1, 4, 0, 1, 1),
 (16, 16, 2, 4, 1, 1, 4),
 (4, 4, 1, 0, 2, 2, 1),
 (4, None, 1, 4, 2, 1, 4),
 (16, 4, 1, 1, 2, 1, 2),
 (8, 8, 4, 2, 4, 1, 4),
 (None, None, 4, 2, 2, 2, 1),
 (16, 16, 1, 0, 2, 2, 0),
 (16, 4, 1, 2, 1, 2, 1),
 (16, 4, 1, 1, 0, 1, 2),
 (4, None, 0, 4, 1, 4, 0),
 (4, 8, 0, 1, 0, 2, 0),
 (8, 8, 0, 4, 1, 0, 0),
 (4, 8, 2, 1, 0, 4, 0),
 (4, None, 2, 4, 0, 1, 1),
 (None, None, 1, 2, 0, 2, 1),
 (16, 4, 2, 4, 2, 1, 4),
 (16, 64, 2, 1, 4, 0, 0),
 (16, 4, 2, 2, 4, 2, 1),
 (32, 8, 0, 4, 1, 1, 4),
 (16, 16, 1, 4, 2, 1, 2),
 (8, 16, 4, 0, 1, 0, 4),
 (4, 16, 2, 4, 0, 0, 1),
 (32, 16, 1, 2, 0, 2, 2),
 (32, 4, 2, 4, 2, 2, 1),
 (None, None, 2, 4, 1, 1, 1),
 (16, 4, 4, 4, 1, 1, 4),
 (64, None, 0, 1, 2, 2, 1)]